In [ ]:
#Code to find the Stackelberg Equilibria of a Pricing Game

In [8]:


import numpy as np
from scipy.optimize import linprog

from operator import add, neg



def precise_stackelberg_equilibrium_old(leader_payoff_matrix, follower_payoff_matrix):
  num_leader_actions = leader_payoff_matrix.shape[0]
  num_follower_actions = leader_payoff_matrix.shape[1]

  # Define the constraint matrix A_ub and the right-hand side b_ub
  A_ub = np.zeros((num_follower_actions + 2, num_leader_actions))
  b_ub = np.zeros(num_follower_actions + 2)
  bounds = [(0, 1) for _ in range(num_leader_actions)]
  best_leader_payoff = np.min(leader_payoff_matrix)
  best_leader_strategy = np.zeros(num_leader_actions)

  for benchmark_follower_action in range(num_follower_actions):
    row_index = 0
    c_leader = -leader_payoff_matrix[:, benchmark_follower_action]  # maximize leader's payoff when follower plays a particular action
    for i in range(num_follower_actions):
      A_ub[row_index] = follower_payoff_matrix.T[i] - follower_payoff_matrix.T[benchmark_follower_action]
      b_ub[row_index] = 0
      row_index = row_index + 1

    A_ub[row_index] = np.ones(num_leader_actions)
    b_ub[row_index] = 1
    row_index = row_index + 1
    A_ub[row_index] = -1 * np.ones(num_leader_actions)
    b_ub[row_index] = -1

    # Solve the linear programming problem
    result = linprog(c_leader, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
    print(result.status)

    # Extract the solution
    leader_optimal_strategy = result.x

    # compare different LPs for different optimizer actions
    follower_benchmark_distribution = np.zeros(num_follower_actions)
    follower_benchmark_distribution[benchmark_follower_action] = 1
    leader_payoff = evaluate_leader_payoff(leader_payoff_matrix, leader_optimal_strategy, follower_benchmark_distribution)
    if (leader_payoff >= best_leader_payoff):
      best_leader_payoff = leader_payoff
      best_leader_strategy = leader_optimal_strategy
      follower_response = benchmark_follower_action
  return best_leader_payoff, best_leader_strategy, follower_response


def precise_stackelberg_value(leader_payoff_matrix, follower_payoff_matrix):
    (val,_,_) = precise_stackelberg_equilibrium(leader_payoff_matrix, follower_payoff_matrix)
    return val

# code for computing mnse from https://github.com/sid230798/Game_Theory/blob/master/Problem3/analyse_equilibrium.py
def msne(a):
    a = a.T
    ## One zero array for later (z, x)
    ess = np.ones(a.shape[0]+1)
    ess[0] = 0

    c = -1*(1-ess)  ##[-1, 0 ,0 ,0] -1 coeff for z and 0 for x (Max z == min(-z))
    A_ub = np.concatenate((np.ones((1, a.shape[1])), -1*a), axis=0).T
    B_ub = np.zeros(a.shape[1])
    A_eq = np.expand_dims(ess, axis=0)
    B_eq = np.ones(1)
    bounds = [(None, None)] + [(0,1)]*a.shape[0]
    result = linprog(c, A_ub=A_ub, b_ub=B_ub, A_eq=A_eq, b_eq=B_eq, bounds=bounds)
    p1_val, p1_distribution = result.x[0], result.x[1:]

    ## For 2nd player distribution
    ess = np.ones(a.shape[1]+1)
    ess[0] = 0
    c = (1-ess)
    A_ub = np.concatenate((-1*np.ones((a.shape[0], 1)), a), axis=1)
    B_ub = np.zeros(a.shape[0])
    A_eq = np.expand_dims(ess, axis=0)
    A_eq = np.concatenate((A_eq, 1-A_eq), axis=0)
    B_eq = np.array([1, p1_val]) ## Dual Principle w* = z*
    bounds = [(None, None)] + [(0,1)]*a.shape[1]
    result = linprog(c, A_ub=A_ub, b_ub=B_ub, A_eq=A_eq, b_eq=B_eq, bounds=bounds)
    p2_val, p2_distribution = result.x[0], result.x[1:]

    print("MSNE are : {", tuple(p1_distribution), "," ,tuple(p2_distribution), "}")

def maxmin(a):
    a = a.T
    print(a)
    ## One zero array for later (z, x)
    ess = np.ones(a.shape[0]+1)
    ess[0] = 0

    c = -1*(1-ess)  ##[-1, 0 ,0 ,0] -1 coeff for z and 0 for x (Max z == min(-z))
    A_ub = np.concatenate((np.ones((1, a.shape[1])), -1*a), axis=0).T
    B_ub = np.zeros(a.shape[1])
    A_eq = np.expand_dims(ess, axis=0)
    B_eq = np.ones(1)
    bounds = [(None, None)] + [(0,1)]*a.shape[0]
    print("data:")
    print(c)
    print(A_ub)
    print(B_ub)
    print(A_eq)
    print(B_eq)
    print(bounds)
    result = linprog(c, A_ub=A_ub, b_ub=B_ub, A_eq=A_eq, b_eq=B_eq, bounds=bounds)
    p1_distribution = result.x[1:]
    return p1_distribution

def transform_game_matrix(game_matrix, mixed_strategies):
    """
    Transforms each game matrix based on the set of mixed strategies. Each mixed strategy
    becomes a new 'pure' strategy in the transformed games for the followers.
    """
    # Convert the distributions list into a NumPy array for easier manipulation
    distributions_array = np.array(mixed_strategies)
    transformed_matrix = np.dot(game_matrix.T, distributions_array.T).T
    return transformed_matrix


def evaluate_leader_payoff_old(game_matrix, leader_strategy, follower_strategy):
    """
    Evaluates the leader's expected payoff on a certain follower game given the leader's strategy and the follower's strategy.
    """
    leader_strategy = np.array(leader_strategy)
    follower_strategy = np.array(follower_strategy)
    expected_payoff = 0
    for leader_strategy, leader_prob in enumerate(leader_strategy):
        for follower_action in range(len(follower_strategy)):
          expected_payoff += leader_prob * follower_strategy[follower_action] * game_matrix[leader_strategy][follower_action]
    return expected_payoff








In [9]:
def evaluate_leader_payoff(game_matrix, leader_strategy, follower_strategy):
    """
    Evaluates the leader's expected payoff given the leader's strategy and the follower's strategy.
    
    Parameters:
    game_matrix (2D numpy array): The leader's payoff matrix
    leader_strategy (1D numpy array): The leader's mixed strategy
    follower_strategy (1D numpy array): The follower's mixed strategy
    
    Returns:
    float: The expected payoff for the leader
    """
    leader_strategy = np.array(leader_strategy)
    follower_strategy = np.array(follower_strategy)
    
    expected_payoff = 0.0
    
    for i in range(len(leader_strategy)):
        for j in range(len(follower_strategy)):
            # Probability of this outcome
            prob = leader_strategy[i] * follower_strategy[j]
            
            # Payoff for the leader in this outcome
            payoff = game_matrix[i][j]
            
            # Add to the expected payoff
            expected_payoff += prob * payoff
    
    return expected_payoff

In [7]:
def precise_stackelberg_equilibrium(leader_payoff_matrix, follower_payoff_matrix):
    num_leader_actions = leader_payoff_matrix.shape[0]
    num_follower_actions = leader_payoff_matrix.shape[1]

    print(f"Number of leader actions: {num_leader_actions}")
    print(f"Number of follower actions: {num_follower_actions}")

    # Define the constraint matrix A_ub and the right-hand side b_ub
    A_ub = np.zeros((num_follower_actions + 2, num_leader_actions))
    b_ub = np.zeros(num_follower_actions + 2)
    bounds = [(0, 1) for _ in range(num_leader_actions)]
    best_leader_payoff = float('-inf')  # Initialize to negative infinity
    best_leader_strategy = np.zeros(num_leader_actions)
    follower_response = None

    for benchmark_follower_action in range(num_follower_actions):
        print(f"\nChecking follower action {benchmark_follower_action}")
        row_index = 0
        c_leader = -leader_payoff_matrix[:, benchmark_follower_action]  # maximize leader's payoff when follower plays a particular action
        for i in range(num_follower_actions):
            A_ub[row_index] = follower_payoff_matrix.T[i] - follower_payoff_matrix.T[benchmark_follower_action]
            b_ub[row_index] = 0
            row_index += 1

        A_ub[row_index] = np.ones(num_leader_actions)
        b_ub[row_index] = 1
        row_index += 1
        A_ub[row_index] = -1 * np.ones(num_leader_actions)
        b_ub[row_index] = -1

        print("Linear programming setup:")
        print(f"c_leader: {c_leader}")
        print(f"A_ub:\n{A_ub}")
        print(f"b_ub: {b_ub}")
        print(f"bounds: {bounds}")

        # Solve the linear programming problem
        result = linprog(c_leader, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
        print(f"LP result status: {result.status}")

        if result.success:
            # Extract the solution
            leader_optimal_strategy = result.x
            print(f"Leader optimal strategy: {leader_optimal_strategy}")

            # compare different LPs for different optimizer actions
            follower_benchmark_distribution = np.zeros(num_follower_actions)
            follower_benchmark_distribution[benchmark_follower_action] = 1
            leader_payoff = np.dot(leader_optimal_strategy, leader_payoff_matrix[:, benchmark_follower_action])
            print(f"Leader payoff: {leader_payoff}")

            if leader_payoff > best_leader_payoff:
                best_leader_payoff = leader_payoff
                best_leader_strategy = leader_optimal_strategy
                follower_response = benchmark_follower_action
        else:
            print(f"Linear programming failed for follower action {benchmark_follower_action}")

    print(f"\nFinal results:")
    print(f"Best leader payoff: {best_leader_payoff}")
    print(f"Best leader strategy: {best_leader_strategy}")
    print(f"Follower response: {follower_response}")

    return best_leader_payoff, best_leader_strategy, follower_response

In [10]:
#Prisoners dilemma game matrix

leader_matrix = np.array([[3, 0],
              [5, 1]])
follower_matrix = np.array([[3, 5],
              [0, 1]])

best_leader_payoff, best_leader_strategy, follower_response = precise_stackelberg_equilibrium(leader_matrix, follower_matrix)

Number of leader actions: 2
Number of follower actions: 2

Checking follower action 0
Linear programming setup:
c_leader: [-3 -5]
A_ub:
[[ 0.  0.]
 [ 2.  1.]
 [ 1.  1.]
 [-1. -1.]]
b_ub: [ 0.  0.  1. -1.]
bounds: [(0, 1), (0, 1)]
LP result status: 2
Linear programming failed for follower action 0

Checking follower action 1
Linear programming setup:
c_leader: [ 0 -1]
A_ub:
[[-2. -1.]
 [ 0.  0.]
 [ 1.  1.]
 [-1. -1.]]
b_ub: [ 0.  0.  1. -1.]
bounds: [(0, 1), (0, 1)]
LP result status: 0
Leader optimal strategy: [-0.  1.]
Leader payoff: 1.0

Final results:
Best leader payoff: 1.0
Best leader strategy: [-0.  1.]
Follower response: 1


In [11]:


# Battle of the Sexes (with a twist)
leader_matrix = np.array([[3, 0],
                          [0, 2]])
follower_matrix = np.array([[2, 0],
                            [0, 3]])

best_leader_payoff, best_leader_strategy, follower_response = precise_stackelberg_equilibrium(leader_matrix, follower_matrix)

print("Battle of the Sexes (with a twist)")
print(f"Best leader payoff: {best_leader_payoff}")
print(f"Best leader strategy: {best_leader_strategy}")
print(f"Follower response: {follower_response}")
print(f"Expected: Payoff 3, Strategy [1, 0], Response 0")

Number of leader actions: 2
Number of follower actions: 2

Checking follower action 0
Linear programming setup:
c_leader: [-3  0]
A_ub:
[[ 0.  0.]
 [-2.  3.]
 [ 1.  1.]
 [-1. -1.]]
b_ub: [ 0.  0.  1. -1.]
bounds: [(0, 1), (0, 1)]
LP result status: 0
Leader optimal strategy: [1. 0.]
Leader payoff: 3.0

Checking follower action 1
Linear programming setup:
c_leader: [ 0 -2]
A_ub:
[[ 2. -3.]
 [ 0.  0.]
 [ 1.  1.]
 [-1. -1.]]
b_ub: [ 0.  0.  1. -1.]
bounds: [(0, 1), (0, 1)]
LP result status: 0
Leader optimal strategy: [-0.  1.]
Leader payoff: 2.0

Final results:
Best leader payoff: 3.0
Best leader strategy: [1. 0.]
Follower response: 0
Battle of the Sexes (with a twist)
Best leader payoff: 3.0
Best leader strategy: [1. 0.]
Follower response: 0
Expected: Payoff 3, Strategy [1, 0], Response 0


In [12]:


# Chicken Game
leader_matrix = np.array([[3, 2],
                          [4, 1]])
follower_matrix = np.array([[3, 4],
                            [2, 1]])

best_leader_payoff, best_leader_strategy, follower_response = precise_stackelberg_equilibrium(leader_matrix, follower_matrix)

print("Chicken Game")
print(f"Best leader payoff: {best_leader_payoff}")
print(f"Best leader strategy: {best_leader_strategy}")
print(f"Follower response: {follower_response}")
print(f"Expected: Payoff 4, Strategy [0, 1], Response 0")

Number of leader actions: 2
Number of follower actions: 2

Checking follower action 0
Linear programming setup:
c_leader: [-3 -4]
A_ub:
[[ 0.  0.]
 [ 1. -1.]
 [ 1.  1.]
 [-1. -1.]]
b_ub: [ 0.  0.  1. -1.]
bounds: [(0, 1), (0, 1)]
LP result status: 0
Leader optimal strategy: [-0.  1.]
Leader payoff: 4.0

Checking follower action 1
Linear programming setup:
c_leader: [-2 -1]
A_ub:
[[-1.  1.]
 [ 0.  0.]
 [ 1.  1.]
 [-1. -1.]]
b_ub: [ 0.  0.  1. -1.]
bounds: [(0, 1), (0, 1)]
LP result status: 0
Leader optimal strategy: [1. 0.]
Leader payoff: 2.0

Final results:
Best leader payoff: 4.0
Best leader strategy: [-0.  1.]
Follower response: 0
Chicken Game
Best leader payoff: 4.0
Best leader strategy: [-0.  1.]
Follower response: 0
Expected: Payoff 4, Strategy [0, 1], Response 0


In [13]:


# Inspection Game 
leader_matrix = np.array([[1, -1],
                          [3, -3]])
follower_matrix = np.array([[0, -3],
                            [-1, 1]])

best_leader_payoff, best_leader_strategy, follower_response = precise_stackelberg_equilibrium(leader_matrix, follower_matrix)

print("Inspector Game (properly corrected)")
print(f"Best leader payoff: {best_leader_payoff}")
print(f"Best leader strategy: {best_leader_strategy}")
print(f"Follower response: {follower_response}")
print(f"Expected: Payoff 2.2, Strategy [0.4, 0.6], Response 0")

Number of leader actions: 2
Number of follower actions: 2

Checking follower action 0
Linear programming setup:
c_leader: [-1 -3]
A_ub:
[[ 0.  0.]
 [-3.  2.]
 [ 1.  1.]
 [-1. -1.]]
b_ub: [ 0.  0.  1. -1.]
bounds: [(0, 1), (0, 1)]
LP result status: 0
Leader optimal strategy: [0.4 0.6]
Leader payoff: 2.2

Checking follower action 1
Linear programming setup:
c_leader: [1 3]
A_ub:
[[ 3. -2.]
 [ 0.  0.]
 [ 1.  1.]
 [-1. -1.]]
b_ub: [ 0.  0.  1. -1.]
bounds: [(0, 1), (0, 1)]
LP result status: 0
Leader optimal strategy: [0.4 0.6]
Leader payoff: -2.2

Final results:
Best leader payoff: 2.2
Best leader strategy: [0.4 0.6]
Follower response: 0
Inspector Game (properly corrected)
Best leader payoff: 2.2
Best leader strategy: [0.4 0.6]
Follower response: 0
Expected: Payoff 2.2, Strategy [0.4, 0.6], Response 0
